In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utilsCM
import csv


In [2]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../code-00-preprocessdataset/'
W2S = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
Y_embeddings = W2S.values[:,1:W2S.shape[1]-1].astype(np.float) #just for threshold

tresh_bonf = utilsCM.p2r(.05/Y_embeddings.shape[1], 1470)

In [3]:
datapath = '../../../data-04/'
nsample = 12
WrdThingsInfo = pd.read_csv(datapath + 'KeptTHINGSInfo_n' + str(nsample) +'.csv',sep=',',index_col = 0)

In [4]:
# keyword = {'ROIpred'}
keyword = {'DNNActvtn','ROIpred'}

layer =  {'conv_1','conv_5','fc_3'}

ROI = {'EVC', 'ObjectROI'}

# Sub = [1,2,3,4]
Sub = [1,2,3,4]

#how many components to keep?
Keepncomps = [20]
pretrained_val = True

datapath = '../../../data-05/'
savepath = '../../../data-06/'

In [5]:
W2S = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",").set_index('item').drop(labels='sense2251', axis=1)
W2S_subset = W2S.iloc[WrdThingsInfo.old_index]

In [6]:
W2S_subset

,sense001,sense002,sense003,sense004,sense005,sense006,sense007,sense008,sense009,sense010,...,sense2241,sense2242,sense2243,sense2244,sense2245,sense2246,sense2247,sense2248,sense2249,sense2250
item,,,,,,,,,,,,,,,,,,,,,
ambulance,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.005953,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ant,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
bag,0.003663,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
bandage,0.000000,0.000000,0.0,0.000000,0.0,0.007127,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
bassoon,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weasel,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
wedge,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
wine,0.000000,0.007286,0.0,0.006826,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [12]:
##Showing histograms for senses above threshold
myDict = {}

for ilayer in layer:
    
    for ikeyword in keyword:        
        
        for icomps in Keepncomps:
            thisPrediction = []
            
            if ikeyword is 'DNNActvtn':
                if not pretrained_val:
                    filename = 'PredictSENSES_' + ikeyword + '_'+ ilayer + '_'+ str(icomps) +'PCs_untrained'
                    csvfilename = savepath + ikeyword+ "_" + ilayer + '_'+ str(icomps) +'PCs_untrained.csv'
                else:
                    filename = 'PredictSENSES_' + ikeyword + '_'+ ilayer + '_'+ str(icomps) +'PCs'
                    csvfilename = savepath + ikeyword+ "_" + ilayer + '_'+ str(icomps) +'PCs.csv'
                    
                thisPrediction = np.load(datapath + filename + '.npy')
                
                vecWithNan = np.argsort(thisPrediction)[::-1]#first 10 senses indeces, from highest to least hgih
                sortedIndeces = vecWithNan[~np.isnan(thisPrediction[vecWithNan])][0:9]
                
                Sample_W2S = W2S_subset.iloc[:,sortedIndeces]
                WeightedWords_rows = (Sample_W2S != 0).any(axis=1)#finds rows of zeros for those tops senses
                new_W2S = W2S_subset.loc[WeightedWords_rows] #takes them off from dataset
                
                ##Goes through each sense, prints out the top ten words and save them in a csv
                listOfWeights = []
                count = 0  
                for i in sortedIndeces:
                    count += 1
                    listOfWeights.append("Top "+ str(count) + " Sense " + str(i))
                    listOfWeights.append(new_W2S.iloc[:,i].sort_values(0)[::-1][0:9])
                    
                    
                
                #save in csv file
                with open(csvfilename, 'w', newline='') as csvfile:
                    wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
                    wr.writerow(listOfWeights)
            
            
            elif ikeyword is 'ROIpred':
                
                for iROI in ROI:
                    thisPrediction = []
                    if not pretrained_val:
                        filename = 'PredictSENSES_' + ikeyword + '_' +iROI + '_'+ ilayer + '_'+ str(icomps) +'PCs_untrained'
                        csvfilename = savepath + ikeyword + "_" + iROI +'_' + ilayer + '_'+ str(icomps) +'PCs_untrained.csv'
                    else:
                        filename = 'PredictSENSES_' + ikeyword + '_' +iROI + '_'+ ilayer + '_'+ str(icomps) +'PCs'
                        csvfilename = savepath + ikeyword + "_" + iROI +'_' + ilayer + '_'+ str(icomps) +'PCs.csv'

                    
                    thisPrediction = np.load(datapath + filename + '.npy')
                    
                    vecWithNan = np.argsort(thisPrediction)[::-1] #first 10 senses indeces, from highest to least hgih
                    sortedIndeces = vecWithNan[~np.isnan(thisPrediction[vecWithNan])]
                    
                    Sample_W2S = W2S_subset.iloc[:,sortedIndeces]#eliminate rows of zeros
                    WeightedWords_rows = (Sample_W2S != 0).any(axis=1)
                    new_W2S = W2S_subset.loc[WeightedWords_rows] #takes them off from dataset

                    listOfWeights = []
                    count = 0
                    for i in sortedIndeces:
                        count += 1
                        listOfWeights.append("Top "+ str(count) + " Sense " + str(i))
                        listOfWeights.append(new_W2S.iloc[:,i].sort_values(0)[::-1][0:9])
                                    
                    with open(csvfilename, 'w', newline='') as csvfile:
                        wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(listOfWeights)


In [13]:
sortedIndeces

array([ 933, 2133, 1894, ..., 1440, 1978,  598])

In [17]:
vec=np.argsort(thisPrediction)[::-1]

array([ 431, 1104,  489, ..., 2209, 1515,  852])